In [ ]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
import pprint

In [ ]:
df = pd.read_csv('./data/filevine_case_study_justice - train set_source data set.csv')

def find_tags(txt):
    soup = BeautifulSoup(txt, 'html.parser')
    tags =[tag.name for tag in soup.find_all()]
    return tags

def clean_text(txt):
    soup = BeautifulSoup(txt.strip(), 'html.parser')
    clean_text = soup.get_text()
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
    clean_text = clean_text.lower()
    return clean_text

df['facts'].apply(find_tags).to_csv('tags.csv')

# print(df)

df['facts_clean'] = df['facts'].apply(clean_text)

# print(df['facts_clean'])

# df_dictionary = df[['href', 'facts_clean']].to_dict(orient='records')

# print(df_dictionary)

df[['href', 'facts_clean']]

In [ ]:
df[df['facts_clean'].str.contains("currently unavailable")]

In [ ]:
df[df['facts_clean'].str.contains("currently unknown")]

In [ ]:
clean_df = df[['href', 'facts_clean']].to_dict('records')


In [ ]:
type(df[['href', 'facts_clean']].rename(columns={"href": "_id"}).('records'))

In [ ]:
print(clean_df)

In [ ]:
type(df_dictionary)

In [ ]:
# mq = marqo.Client(url="http://localhost:8882")

# mq.create_index("my-first-index", model="hf/e5-base-v2")

# mq.index("my-first-index").add_documents(
#     [
#         {
#             "Title": "The Travels of Marco Polo",
#             "Description": "A 13th-century travelogue describing Polo's travels",
#         },
#         {
#             "Title": "Extravehicular Mobility Unit (EMU)",
#             "Description": "The EMU is a spacesuit that provides environmental protection, "
#             "mobility, life support, and communications for astronauts",
#             "_id": "article_591",
#         },
#     ],
#     tensor_fields=["Description"],
# )

# results = mq.index("my-first-index").search(
#     q="What is the best outfit to wear on the moon?"
# )

# print=

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

mq.create_index("test", model="hf/multilingual-e5-large")

mq.index("test").add_documents(
    clean_df,
    tensor_fields=["facts_clean"],
    client_batch_size=50,
    auto_refresh=False
)

In [ ]:
# results = mq.index("test").search(
#     q="he city of Renton, Washington, enacted a zoning ordinance that prohibited adult motion picture",
#     filter_string=['facts_clean']
# )
query = "due process"
results = mq.index("test").search(query, search_method='TENSOR')
print(results)


# Prints the nicely formatted dictionary
pprint.pprint(results)

In [ ]:
mq.index("test").get_stats()

In [ ]:
mq.index("my-first-index").get_documents(
    expose_facets=True,
    document_ids=[]
)

In [ ]:
mq.index("sentences_test").delete()

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

settings = {
    "textPreprocessing": {
        "splitLength": 2,
        "splitOverlap": 0,
        "splitMethod": "sentence",
    },
}

mq.create_index("sentences_test", 
                model="hf/multilingual-e5-large",
                settings_dict=settings
)

mq.index("sentences_test").add_documents(
    clean_df,
    tensor_fields=["facts_clean"],
    client_batch_size=50
)

In [ ]:
mq.index('sentences_test').get_stats()

In [ ]:
# results = mq.index("test").search(
#     q="he city of Renton, Washington, enacted a zoning ordinance that prohibited adult motion picture",
#     filter_string=['facts_clean']
# )
query = "A New Jersey law prohibited the importation of most solid or liquid waste which originated or was collected outside the territorial limits of the State."
results = mq.index("sentences_test").search(query, search_method='TENSOR')

# Prints the nicely formatted dictionary
pprint.pprint(results)

In [ ]:
pd.DataFrame(results['hits']).loc[1]._highlights
